In [4]:
import scanpy, scprep
import pandas as pd
import numpy as np
import phate

import sys
sys.path.append('..')
from run.run_gspa import calculate_wavelet_dictionary
from run.run_ae_default_config import run_ae
from utils import *
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
import leidenalg
import graphtools
import networkx as nx
from scipy.stats import spearmanr
import magic
from scipy.spatial.distance import pdist, squareform

/home/aarthivenkat/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [7]:
data_int = scanpy.read_h5ad('data/int.h5ad')
data_gc = scanpy.read_h5ad('data/GC.h5ad')
hits = data_gc.to_df()
hits.columns = [x.split('-')[0] for x in data_gc.var_names]
hits = hits.groupby(axis=1, level=0).sum()
guide_hit_signal = hits.columns[np.where(hits.values > 0)[1]]
guides = hits.columns

/home/aarthivenkat/.local/lib/python3.8/site-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [3]:
embeddings = {}
for guide in guides:
    genes = np.load(f'results/{guide}_GSPA.npz', allow_pickle=True)['genes']
    embeddings[guide] = pd.DataFrame(np.load(f'results/{guide}_GSPA.npz')['signal_embedding'], index=genes)

In [4]:
localizations = {}
for guide in guides:
    genes = np.load(f'results/{guide}_GSPA.npz', allow_pickle=True)['genes']
    localizations[guide] = pd.DataFrame(np.load(f'results/{guide}_GSPA.npz')['localization_score'], index=genes)

## Edges in perturbation that are not in normal

In [8]:
for guide in guides:
    print (guide)
    threshold = np.percentile(localizations['NEG'], q=75)
    neg_loc = embeddings['NEG'].loc[(localizations['NEG'] > threshold)[0]]
    
    threshold = np.percentile(localizations[guide], q=75)
    KO_loc = embeddings[guide].loc[(localizations[guide] > threshold)[0]]
    
    loc_genes_both = set(neg_loc.index).intersection(KO_loc.index)
    
    G = graphtools.Graph(embeddings['NEG'].loc[loc_genes_both], knn=5, use_pygsp=True)
    G_neg = G.A.todense().astype(int)
    
    G = graphtools.Graph(embeddings[guide].loc[loc_genes_both], knn=100, use_pygsp=True)
    G_KO = G.A.todense().astype(int)
    
    A = G_neg - G_KO
    A = A > 0 # edges that are in sparser neg graph and not in more complete KO graph
    G_neg_only = nx.Graph(A)
    G_neg_only = nx.relabel_nodes(G_neg_only, dict(zip(range(G_neg_only.number_of_nodes()), list(loc_genes_both))))
    
    nx.write_edgelist(G_neg_only, f'results/{guide}_only_graph.tsv', delimiter='\t', data=False)
    
    for cc in nx.connected_components(G_neg_only):
        if len(cc) > 1:
            print (cc)

BACH2
{'Cblb', 'Gm11579', 'F10', 'Maml2'}
{'Lgals3', 'AI506816'}
{'Pim1', 'Trac', 'Lcn4'}
{'Tnfrsf9', 'Cotl1'}
BATF
{'Rps2', 'Tox'}
{'Rpsa', 'Fchsd2'}
{'Tent5a', 'Gm11579'}
BCL6
{'Dzank1', 'Ifitm3', 'Fmr1nb', 'Adam19', 'Lsp1', 'Nccrp1', 'Klf1'}
{'Tagln2', 'Cmtm7'}
{'Cd5', 'Cd69', 'Apobec3'}
{'Samd13', 'Msc', 'Rdm1', 'Hdgf', 'Cfl1'}
{'Rpl32', 'Itga4'}
{'Ssbp2', 'Zeb1'}
CXCL13
{'Dzank1', 'Cd69', 'Adam19', 'Ms4a4c', 'Apobec3', 'Lsp1', '2310001H17Rik', 'Klf1', 'Trac'}
{'Arhgap15', 'Cblb', 'Ifngr1', 'Gm11579', 'F10', 'Pim1'}
{'Ptma', 'Stmn1'}
{'Dynll1', 'Hsp90b1'}
{'Gm8369', 'Gimap6', 'Rpl12', 'Slfn1'}
{'Neurl3', 'Sntb1', 'Rpsa'}
{'Rcbtb2', 'Trbc1'}
CXCR5
{'Maml2', 'Sdcbp2', 'Fmr1nb', 'Id2', 'Gm11579', 'Gzmm', 'F10', 'Pim1'}
{'Gapdh', 'Tagln2', 'Cmtm7', 'CAAA01147332.1'}
{'Tnfrsf9', 'Rom1', 'Gm29666'}
{'Ptma', 'Lmnb1', 'Hmgn2', 'Cep55'}
{'Gm8369', 'Mbnl1'}
DNMT1
{'Gm4316', '3830403N18Rik'}
{'Rdm1', 'H1fx'}
DNMT3A
{'Rgs10', 'Rpsa'}
{'Fabp5', 'Rad21', 'Samd13', 'Hnrnpa2b1'}
DNMT3B
{'Trac', 'C